In [3]:
import psycopg2
import pandas as pd
from string import Template

In [19]:
dataname = "small_adult_1"

In [20]:
index = "1"

### Connect to db

In [21]:
conn = psycopg2.connect(host="localhost",
                        database="{}_{}".format(dataname,index), 
                        user="holocleanuser", 
                        password="abcd1234")

In [22]:
def query_df(q, value = []):
    if len(value) != 0:
        df = pd.read_sql_query(q, conn, params=value)
    else:
        df = pd.read_sql_query(q, conn)
    return df

In [23]:
def table_lists():
    df = query_df("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
    return df

In [24]:
table_lists()

,table_name
0,small_adult_1
1,dk_cells
2,cell_domain
3,pos_values
4,cell_distr
5,inf_values_idx
6,inf_values_dom
7,small_adult_1_repaired
8,small_adult_1_clean


In [25]:
# get all attributes 
data = query_df("select * from {}".format(dataname))
attributes = data.columns.values[1:]
attributes

array(['age', 'workclass', 'education', 'maritalstatus', 'occupation',
       'relationship', 'race', 'sex', 'hoursperweek', 'country', 'income'],
      dtype=object)

### Total Errors

In [26]:
errors_template = Template('SELECT t1._tid_, t2._attribute_, t1.$attr as init, t2._value_ as gt '\
                            'FROM $init_table as t1, $grdt_table as t2 '\
                            'WHERE t1._tid_ = t2._tid_ '\
                              'AND t2._attribute_ = \'$attr\' '\
                              'AND t1.$attr != t2._value_')

In [27]:
def get_total_errors():
    all_error = []
    for attr in attributes:
        query = errors_template.substitute(init_table=dataname, grdt_table='%s_clean'%dataname,
                        attr=attr)
        df = query_df(query)
        all_error.append(df)
    return all_error

In [28]:
all_error = get_total_errors()

In [29]:
all_error = pd.concat(all_error, ignore_index=True)

In [31]:
all_error

,_tid_,_attribute_,init,gt
0,1,workclass,?,_nan_
1,4,workclass,?,_nan_
2,11,workclass,?,_nan_
3,12,workclass,?,_nan_
4,17,workclass,?,_nan_
5,38,workclass,?,_nan_
6,41,workclass,?,_nan_
7,42,workclass,?,_nan_
8,43,workclass,?,_nan_
9,45,workclass,?,_nan_


In [16]:
init_count = all_error.shape[0]
init_count

1407

### Total Repair

In [35]:
query = "SELECT t1._tid_, t1.attribute, t1.init_value as init, t2.rv_value as repair " \
                 "FROM %s as t1, %s as t2 " \
                 "WHERE t1._tid_ = t2._tid_ " \
                   "AND t1.attribute = t2.attribute " \
                   "AND t1.init_value != t2.rv_value"\
                %('cell_domain', 'inf_values_dom')
all_repair = query_df(query)

In [36]:
all_repair.head()

,_tid_,attribute,init,repair
0,5,num_persons_worked_for_employer,x,0
1,28,weeks_worked_in_year,48,52
2,48,weeks_worked_in_year,40,52
3,55,detailed_household_and_family_stat,nonfamily householder,householder
4,65,num_persons_worked_for_employer,5,4


In [37]:
total_repair_init_count = all_repair.shape[0]
total_repair_init_count

29

In [39]:
query = "SELECT t1._tid_, t1.attribute, t1.init_value as init, t2.rv_value as repair " \
         "FROM %s as t1, %s as t2, %s as t3 " \
         "WHERE t1._tid_ = t2._tid_ " \
           "AND t1.attribute = t2.attribute " \
           "AND t1.init_value != t2.rv_value " \
           "AND t1._tid_ = t3._tid_ " \
           "AND t1.attribute = t3._attribute_"\
        %('cell_domain', 'inf_values_dom', '%s_clean'%dataname)
repair_gt = query_df(query)

In [40]:
repair_gt.head()

,_tid_,attribute,init,repair
0,5,num_persons_worked_for_employer,x,0
1,28,weeks_worked_in_year,48,52
2,48,weeks_worked_in_year,40,52
3,55,detailed_household_and_family_stat,nonfamily householder,householder
4,65,num_persons_worked_for_employer,5,4


In [41]:
repair_gt_init_count = repair_gt.shape[0]
repair_gt_init_count

29

### Correct Repair

In [42]:
repairs_template = Template('SELECT errors._tid_, errors._attribute_, '\
                            ' errors.init, errors._value_ as gt, repairs.rv_value as repair FROM'\
                            '(SELECT t2._tid_, t2._attribute_, t2._value_, t1.$attr as init '\
                             'FROM $init_table as t1, $grdt_table as t2 '\
                             'WHERE t1._tid_ = t2._tid_ '\
                               'AND t2._attribute_ = \'$attr\' '\
                               'AND t1.$attr != t2._value_ ) as errors, $inf_dom as repairs '\
                              'WHERE errors._tid_ = repairs._tid_ '\
                                'AND errors._attribute_ = repairs.attribute ')

In [43]:
def get_total_repair():
    all_rp = []
    for attr in attributes:
        query = repairs_template.substitute(init_table=dataname, grdt_table='%s_clean'%dataname, 
                                            inf_dom = 'inf_values_dom', attr=attr)
        df = query_df(query)
        all_rp.append(df)
    return all_rp

In [44]:
all_repair_gt = get_total_repair()

In [45]:
all_repair_gt = pd.concat(all_repair_gt, ignore_index=True)

In [54]:
all_repair_gt.head(30)

,_tid_,_attribute_,init,gt,repair
0,60,detailed_industry_recode,x,4,x
1,213,detailed_industry_recode,x3,43,x3
2,528,detailed_industry_recode,x,0,0
3,922,detailed_occupation_recode,x,0,0
4,632,detailed_household_and_family_stat,chxld <18 never marr not in subfamily,child <18 never marr not in subfamily,chxld <18 never marr not in subfamily
5,705,detailed_household_and_family_stat,child <18 never marr not in xubfamily,child <18 never marr not in subfamily,child <18 never marr not in xubfamily
6,299,migration_code_move_within_reg,different staxe in west,different state in west,different staxe in west
7,217,migration_prev_res_in_sunbelt,nxt in universe,not in universe,not in universe
8,5,num_persons_worked_for_employer,x,0,0
9,144,num_persons_worked_for_employer,x,0,0


In [47]:
all_repair_gt.shape[0]

14

### Detected Errors

In [48]:
query = "SELECT t1._tid_, t1.init_value as init, t2._value_ as gt " \
        "FROM %s as t1, %s as t2, %s as t3 " \
        "WHERE t1._tid_ = t2._tid_ AND t1._cid_ = t3._cid_ " \
        "AND t1.attribute = t2._attribute_ " \
        "AND t1.init_value != t2._value_" \
        % ('cell_domain', '%s_clean'%dataname, 'dk_cells')

In [49]:
init_error = query_df(query)

In [50]:
init_error.shape[0]

14

In [55]:
# total errors
init_count

33